# Track notebooks, scripts & functions

This guide explains how to use {meth}`~lamindb.track` & {meth}`~lamindb.finish` to track notebook & scripts along with their inputs and outputs.
For tracking pipelines, see: {doc}`docs:pipelines`.

In [ ]:
# !pip install 'lamindb[jupyter]'
!lamin init --storage ./test-track

## Track a notebook or script

Call {meth}`~lamindb.track` to register your notebook or script as a `transform` and start tracking inputs & outputs of a run.

```python
import lamindb as ln

ln.track()  # initiate a tracked notebook/script run

# your code

ln.finish()  # mark run as finished, save execution report, source code & environment
```

<br>

:::{dropdown} Here is how a notebook with run report looks on the hub.

Explore it [here](https://lamin.ai/laminlabs/lamindata/transform/PtTXoc0RbOIq).

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8dJfH.png" width="900px">

:::

:::{dropdown} Assign a `uid` to a notebook or script

If you want to retain _one version history_ when renaming notebooks and scripts, pass a `uid` to `ln.track()`, e.g.

```python
ln.track("9priar0hoE5u0000")
```

<br>

To obtain a `uid` value, copy it from the logging statement (`Transform('9priar0hoE5u0000')`) when running `ln.track()` a first time _without_ passing a `uid`.

:::

## Load a notebook or script

On the hub, search or filter the `transform` page and then load a script or notebook on the CLI. For example,

```bash
lamin load https://lamin.ai/laminlabs/lamindata/transform/13VINnFk89PE
```

## Query a notebook or script

You find your notebooks and scripts in the {class}`~lamindb.Transform` registry (along with pipelines & functions). {class}`~lamindb.Run` stores executions.
You can use all usual ways of querying to obtain one or multiple transform records, e.g.:

```python
transform = ln.Transform.get(key="my_analyses/my_notebook.ipynb")
transform.source_code  # source code
transform.latest_run.report  # report of latest run
transform.latest_run.environment  # environment of latest run
transform.runs  # all runs
```

## Sync scripts with git

To sync with your git commit, add the following line to your script:

```python
ln.settings.sync_git_repo = <YOUR-GIT-REPO-URL>
```

```{eval-rst}
.. literalinclude:: scripts/synced-with-git.py
   :language: python
   :caption: synced-with-git.py
```

:::{dropdown} You'll now see the GitHub emoji clickable on the hub.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/IpV8Kiq4xUbgXhzlUYT7.png" width="900px">
:::

## Track parameters

In addition to tracking source code, run reports & environments, you can easily track run parameters.

(track-run-parameters)=

### Track run parameters

Before tracking parameter values, you need to define valid parameters, e.g.:

In [ ]:
import lamindb as ln

ln.Param(name="input_dir", dtype="str").save()
ln.Param(name="learning_rate", dtype="float").save()
ln.Param(name="preprocess_params", dtype="dict").save()

Upon running the below script without those parameters defined, you'll get a `ValidationError` from which you can copy & paste the definitions.

```{eval-rst}
.. literalinclude:: scripts/run-track-with-params.py
   :language: python
   :caption: run-track-with-params.py
```

Run the script.

In [ ]:
!python scripts/run-track-with-params.py  --input-dir ./mydataset --learning-rate 0.01 --downsample

### Query by run parameters

Query for all runs that match a certain parameters:

In [ ]:
ln.Run.params.filter(
    learning_rate=0.01, input_dir="./mydataset", preprocess_params__downsample=True
).df()

Note that:

* `preprocess_params__downsample=True` traverses the dictionary `preprocess_params` to find the key `"downsample"` and match it to `True`
* nested keys like `"downsample"` in a dictionary do not appear in `Param` and hence, do not get validated

### Access parameters of a run

Below is how you get the parameter values that were used for a given run.

In [ ]:
run = ln.Run.params.filter(learning_rate=0.01).order_by("-started_at").first()
run.params.get_values()

:::{dropdown} Here is how it looks [on the hub](https://lamin.ai/laminlabs/lamindata/transform/JjRF4mACd9m00001).

<img width="500" alt="image" src="https://github.com/user-attachments/assets/d8a5df37-d585-4940-b6f0-91f99b6c436c">

:::

### Explore all parameter values

If you want to query all parameter values across all runs, use {class}`~lamindb.core.ParamValue`.

In [ ]:
ln.core.ParamValue.df(include=["param__name", "created_by__handle"])

In [ ]:
assert run.params.get_values() == {
    "input_dir": "./mydataset",
    "learning_rate": 0.01,
    "preprocess_params": {"downsample": True, "normalization": "the_good_one"},
}

## Manage notebook templates

A notebook acts like a template upon using `lamin load` to load it. Consider you run:

```bash
lamin load https://lamin.ai/account/instance/transform/Akd7gx7Y9oVO0000
```

Upon running the returned notebook, you'll automatically create a new version and be able to browse it via the version dropdown on the UI.

Additionally, you can:

- label using `ULabel`, e.g., `transform.ulabels.add(template_label)`
- tag with an indicative `version` string, e.g., `transform.version = "T1"; transform.save()`

:::{dropdown} Saving a notebook as an artifact

Sometimes you might want to save a notebook as an artifact. This is how you can do it:

```bash
lamin save template1.ipynb --key templates/template1.ipynb --description "Template for analysis type 1" --registry artifact
```

:::

## Track functions

In [ ]:
import lamindb as ln

ln.track()  # track this notebook
ln.Param(name="subset_rows", dtype="int").save()  # define run parameters
ln.Param(name="subset_cols", dtype="int").save()
ln.Param(name="input_artifact_key", dtype="str").save()
ln.Param(name="output_artifact_key", dtype="str").save()

Define a function and decorate it with `tracked()`:

In [ ]:
@ln.tracked()
def subset_dataframe(
    input_artifact_key: str,
    output_artifact_key: str,
    subset_rows: int = 2,
    subset_cols: int = 2,
) -> None:
    artifact = ln.Artifact.get(key=input_artifact_key)
    dataset = artifact.load()
    new_data = dataset.iloc[:subset_rows, :subset_cols]
    ln.Artifact.from_df(new_data, key=output_artifact_key).save()

Run the function with default params:

In [ ]:
df = ln.core.datasets.small_dataset1(otype="DataFrame")
input_artifact_key = "my_analysis/dataset.parquet"
artifact = ln.Artifact.from_df(df, key=input_artifact_key).save()

ouput_artifact_key = input_artifact_key.replace(".parquet", "_subsetted.parquet")
subset_dataframe(input_artifact_key, ouput_artifact_key)

subsetted_artifact = ln.Artifact.get(key=ouput_artifact_key)
subsetted_artifact.view_lineage()

print(subsetted_artifact.run.initiated_by_run.transform)  # notebook
print(subsetted_artifact.run.transform)  # function
print(subsetted_artifact.run)  # run
print(subsetted_artifact.run.params.get_values())  # params
display(subsetted_artifact.run.input_artifacts.df())  # input artifacts
display(subsetted_artifact.run.output_artifacts.df())  # output artifacts

Re-run the function with a different parameter:

In [ ]:
subsetted_artifact = subset_dataframe(
    input_artifact_key, ouput_artifact_key, subset_cols=3
)

subsetted_artifact = ln.Artifact.get(key=ouput_artifact_key)
subsetted_artifact.view_lineage()

print(subsetted_artifact.run.initiated_by_run.transform)  # notebook
print(subsetted_artifact.run.transform)  # function
print(subsetted_artifact.run)  # run
print(subsetted_artifact.run.params.get_values())  # params
display(subsetted_artifact.run.input_artifacts.df())  # input artifacts
display(subsetted_artifact.run.output_artifacts.df())  # output artifacts

See the state of the database:

In [ ]:
ln.view()

### Towards a workflow

```{eval-rst}
.. literalinclude:: scripts/run-workflow.py
   :language: python
   :caption: run-workflow.py
```

In [ ]:
!python scripts/run-workflow.py --subset

In [ ]:
ln.view()

In [ ]:
# clean up test instance
!rm -r ./test-track
!lamin delete --force test-track